In [33]:
import pandas as pd
import datetime
year=['2017','2018','2019','2020']#设置年份
m=['-01','-02','-03','-04','-05','-06','-07','-08','-09','-10','-11','-12']#设置月份
df_indicator=pd.read_csv('daily_basic.csv')
df_indicator_filled=df_indicator.fillna(0)#填充空缺
df_indicator_filled['trade_date']=pd.to_datetime(df_indicator_filled['trade_date'],format='%Y%m%d')#设置时间戳
df_indicator_filled=df_indicator_filled.set_index('trade_date')#设置index为时间序列
df_indicator_filled_list={}#字典保存按年份和月份分割的Dataframe

In [34]:
for i in year:
    month_dict={}
    for j in m:
        month_dict[j]=df_indicator_filled.loc[i+j,:]
    df_indicator_filled_list[i]=month_dict

In [35]:
def get__score(df_basic_filled_list,d,t):#对单一指标进行排序
    df=df_indicator_filled_list
    df=df_indicator_filled_list[t[0:4]][t[4:]]
    df=df.loc[:,['ts_code',d]]
    dg=df.groupby('ts_code')
    dg=dg.mean()
    dg=dg.sort_values(d,ascending=False)
    rank=dg.rank()
    return rank
def unit_socre(df,ind,t):#联合单一指标获得联合指标排序
    tmp=get__score(df,ind[0],t)
    rank=tmp
    for i in range(1,len(ind)):
        tmp=get__score(df,ind[i],t)
        rank = pd.merge(rank,tmp,how='inner',on='ts_code')
    rank['score']=rank.apply(lambda x:x.sum(),axis=1)
    return rank
daily=['pe','pb','ps','total_mv']
score_indicator=unit_socre(df_indicator_filled_list,daily,'2017-04')#获得指定指标序列和指定时间的打分
select=score_indicator.sort_values(by='score',ascending=False).head(100)
print(select.index)

Index(['000503.SZ', '000820.SZ', '300122.SZ', '600233.SH', '002120.SZ',
       '002625.SZ', '600161.SH', '002522.SZ', '000025.SZ', '300136.SZ',
       '600435.SH', '600764.SH', '600876.SH', '600570.SH', '002468.SZ',
       '002346.SZ', '600698.SH', '600610.SH', '300601.SZ', '002797.SZ',
       '000813.SZ', '002127.SZ', '002095.SZ', '600679.SH', '000923.SZ',
       '000068.SZ', '000697.SZ', '300308.SZ', '300482.SZ', '002044.SZ',
       '002451.SZ', '002252.SZ', '002180.SZ', '002569.SZ', '002387.SZ',
       '300101.SZ', '603881.SH', '000831.SZ', '601020.SH', '300085.SZ',
       '300506.SZ', '000766.SZ', '600760.SH', '603986.SH', '002694.SZ',
       '601069.SH', '002622.SZ', '002515.SZ', '300191.SZ', '002530.SZ',
       '300142.SZ', '300558.SZ', '603058.SH', '002676.SZ', '300529.SZ',
       '600393.SH', '002575.SZ', '600771.SH', '300474.SZ', '600158.SH',
       '600779.SH', '603859.SH', '300496.SZ', '300618.SZ', '300302.SZ',
       '600436.SH', '000806.SZ', '300628.SZ', '600110.SH', '6037

In [36]:
l=select.index.to_list()
data=pd.read_csv("company_label.csv")
import re
data["code"]=data["sec_code"].apply(lambda x:re.findall(r"(.+)_.+_.+",x)[0]+"."+re.findall(r".+_(.+)_.+",x)[0])

In [37]:
data1=data[data["code"].isin(l)]
data1["time"]=pd.to_datetime(data1["news_time"])
# data1.to_csv("data1.csv",index=False)

<ipython-input-37-7a7ae9fd029e>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data1["time"]=pd.to_datetime(data1["news_time"])


In [38]:
t=pd.DataFrame(columns=["pos","neg","neu","公司","code"])
for i in data1["code"].unique():
    t1=data1[data1["code"]==i]
    t1.index=data1[data1["code"]==i]["time"]
    t2=t1.resample("1d")["pos","neg","neu"].mean()
    t2["code"]=i
    t2["公司"]=data1[data1["code"]==i].company_name.unique()[0]
    t=pd.concat([t,t2])
t.fillna(0,inplace=True)

In [39]:
import numpy as np
df=pd.read_csv("event_label.csv")
event=df.merge(data1,on="news_id",how="right")

In [40]:
for i in event["event_name"].unique()[1:]:
    event[i]=event["event_name"].apply(lambda x :1 if x==i else 0)
    event[i]=event[i].astype(np.uint8)

In [41]:
ls=list(event.columns[13:])
ls.append("公司")
ls.append("code")
len(event.code.unique())

TypeError: 'numpy.ndarray' object is not callable

In [ ]:
T=pd.DataFrame(columns=ls)
for i in event["code"].unique():
    T1=event[event["code"]==i]
    T1.index=event[event["code"]==i]["time"]
    T2=T1.resample("1d")[list(event.columns[13:])].sum()
    T2["code"]=i
    T2["公司"]=event[event["code"]==i].company_name.unique()[0]
    T=pd.concat([T,T2])
final=pd.concat([t,T.iloc[:,:-2]],axis=1)
final.to_csv("final.csv",index=False)

In [ ]:
final